## ライブラリインストール

In [2]:
import os
import random

import numpy as np
import pandas as pd
# import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import KFold, StratifiedKFold

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.layers import Input, Dense, Conv2D, Activation
from tensorflow.keras.layers import MaxPooling2D, UpSampling2D, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
# from tensorflow.compat.v1.keras.applications import EfficientNetB0 
# tf.compat.v1.keras.applications.efficientnet.EfficientNetB0

In [3]:
def set_randvalue(value):
    # Set a seed value
    seed_value= value 
    # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
    os.environ['PYTHONHASHSEED']=str(seed_value)
    # 2. Set `python` built-in pseudo-random generator at a fixed value
    random.seed(seed_value)
    # 3. Set `numpy` pseudo-random generator at a fixed value
    np.random.seed(seed_value)
    # 4. Set `tensorflow` pseudo-random generator at a fixed value
    tf.random.set_seed(seed_value)

seed_value = 42
set_randvalue(seed_value)

## CSVロード

In [4]:
# train.csvが元のラベルごとにディレクトリに保存されていたデータから作成したcsv
df = pd.read_csv("../data/input/csvs/train.csv")
df["label"] = df["label"].astype(str)
df.head(3)

,image,label
0,img_100026.jpg,0
1,img_10003.jpg,0
2,img_100050.jpg,0


## モデル作成

In [107]:
# EfficientNetB0のモデルを作成
def create_model(weight_flg=False):
    weight = None
    if weight_flg:
        weight = "../model/efficientnetb0_notop.h5" # ImageNetで学習されたモデルをロード
    # include_top=False; 全結合層なし
    base_model = EfficientNetB0(weights=weight, include_top=False, pooling='avg', input_shape=(224,224,3))
    x = Dense(512, activation='relu')(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(10, activation='softmax')(x)
    model = Model(base_model.input, output)
    
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

## 学習

In [5]:
# Hyper parameters
img_size = 224
batch_size = 16
epochs = 250

In [6]:
# Generator作成
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20, # rotation range
    width_shift_range=30/img_size, # 30 pixel
    height_shift_range=30/img_size, # 30 pixel
    zoom_range=0.2
)

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
%%time
# 学習と評価データでのラベルの分布数を一定に保ち学習
skf = StratifiedKFold(n_splits = 5)
splitter = skf.split(df["image"],df["label"])
for i, (train_ids, valid_ids) in enumerate(splitter, 1):
    # データ生成
    train, valid = df.iloc[train_ids], df.iloc[valid_ids]
    train_datagenerator = train_datagen.flow_from_dataframe(
        train,
        directory='../data/input/imgs/train/imgs/',
        x_col='image',
        y_col='label',
        target_size=(img_size, img_size),
        class_mode='categorical',
        batch_size=batch_size,
        seed=seed_value
    )

    valid_datagenerator = valid_datagen.flow_from_dataframe(
        valid,
        directory='../data/input/imgs/train/imgs/',
        x_col='image',
        y_col='label',
        target_size=(img_size, img_size),
        class_mode='categorical',
        batch_size=batch_size,
        seed=seed_value
    )
    
    model = create_model()
    
    # 早期終了
    early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
    
    # 早期終了するのでval_lossが小さいモデルを保存
    model_path = '../model/' +  'Train001_' + "fold" + str(i) + "_best_model.h5"
    checkpoint = ModelCheckpoint(
                    filepath=model_path,
                    monitor='val_loss',
                    save_best_only=True,
                    period=1)
    
    history = model.fit(
        train_datagenerator,
        steps_per_epoch=int(len(train)//batch_size),
        epochs=epochs,
        validation_data=valid_datagenerator,
        validation_steps=int(len(valid)//batch_size),
        verbose=1,
        shuffle=True,
        callbacks=[early_stop, checkpoint])

Found 17939 validated image filenames belonging to 10 classes.
Found 4485 validated image filenames belonging to 10 classes.
Epoch 1/250
1121/1121 [==============================] - 173s 155ms/step - loss: 2.0103 - accuracy: 0.2334 - val_loss: 1.3334 - val_accuracy: 0.4449
Epoch 2/250
1121/1121 [==============================] - 169s 150ms/step - loss: 1.2506 - accuracy: 0.5215 - val_loss: 0.7346 - val_accuracy: 0.7201
Epoch 3/250
1121/1121 [==============================] - 170s 152ms/step - loss: 0.7471 - accuracy: 0.7416 - val_loss: 0.3694 - val_accuracy: 0.8708
Epoch 4/250
1121/1121 [==============================] - 171s 152ms/step - loss: 0.5498 - accuracy: 0.8224 - val_loss: 0.3854 - val_accuracy: 0.8792
Epoch 5/250
1121/1121 [==============================] - 171s 153ms/step - loss: 0.5035 - accuracy: 0.8398 - val_loss: 0.4298 - val_accuracy: 0.8451
Epoch 6/250
1121/1121 [==============================] - 172s 153ms/step - loss: 0.3540 - accuracy: 0.8939 - val_loss: 0.1832 - va

## 推論

In [53]:
model2 = create_model()
model_path = "../Train001_fold" + str(0) + "_best_model.h5"
model2.save(model_path)

In [20]:
submit = pd.read_csv('../data/input/csvs/sample_submission.csv')

In [21]:
test_generator = test_datagen.flow_from_dataframe(
    submit,
    directory='../data/input/imgs/test/',
    x_col='img',
    y_col='c0', # ダミー変数
    target_size=(img_size, img_size),
    class_mode=None,
    batch_size=1,
    verbose=1,
    shuffle=False
)

Found 79726 validated image filenames.


#### 5モデルのうち1モデルで推論

In [44]:
%%time
model_1 = create_model()
model_1.load_weights('../model/Train001_fold1_best_model.h5')
pred_1 = model_1.predict(test_generator)

Wall time: 12min 24s


In [ ]:
pred_df = pd.DataFrame()

In [61]:
columns = submit.columns.values
labels = submit.columns[1:].values

In [62]:
labels

array(['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'],
      dtype=object)

In [63]:
pred_df = pd.DataFrame(columns=columns)

In [64]:
pred_1_df['img'] = submit['img'] 

In [65]:
pred_1_df[labels] = pred_1

In [69]:
pred_1_df.to_csv('../data/output/Train001_fold1_sub.csv', index=False)

#### 5モデルのうち4モデルで推論

In [71]:
model_ = create_model()
model_.load_weights('../model/Train001_fold2_best_model.h5')

In [89]:
columns = submit.columns.values
labels = submit.columns[1:].values

def inference(fold_num):
    model = create_model()
    weight_path = '../model/Train001_fold' + str(fold_num) + '_best_model.h5'
    output_path = '../data/output/Train001_fold' + str(fold_num) + '_sub.csv'
    model.load_weights(weight_path)
    
    pred = model.predict(test_generator, verbose=1)
    pred_df = pd.DataFrame(columns=columns)
    pred_df['img'] = submit['img']
    pred_df[labels] = pred
    pred_df.to_csv(output_path, index=False)

In [90]:
inference(2)
inference(3)
inference(4)
inference(5)

79726/79726 [==============================] - 491s 6ms/step


  0%|                                                                                        | 0/79726 [26:13<?, ?it/s]


79726/79726 [==============================] - 490s 6ms/step


#### アンサンブル：単純平均

In [71]:
submit = pd.read_csv('../data/input/csvs/sample_submission.csv')
columns = submit.columns.values
labels = submit.columns[1:].values

ensemble = 0
for i in range(1,6):
    path = "../data/output/Train001_fold" + str(i) +"_sub.csv"
    ensemble += pd.read_csv(path).values[:,1:] / 5 # fold数で割る

ensemble_df = pd.DataFrame(columns=columns)
ensemble_df['img'] = submit['img']
ensemble_df[labels] = ensemble

ensemble_df.to_csv("../data/output/Train001_ensemble_sub.csv", index=False)

#### アンサンブル：加重平均

fold数ごとのPublic scoreを元に加重平均  
  
 fold: Public score  
 fold1: 0.60428  
 fold2: 0.57041  
 fold3: 0.65702  
 fold4: 0.50968  
 fold5: 0.69935  

In [74]:
fold1_inverse = 1/0.60428  
fold2_inverse = 1/0.57041  
fold3_inverse = 1/0.65702  
fold4_inverse = 1/0.50968  
fold5_inverse = 1/0.69935  

fold_inverse_sum = fold1_inverse + fold2_inverse + fold3_inverse + fold4_inverse + fold5_inverse

fold1_weight = fold1_inverse/(fold_inverse_sum)
fold2_weight = fold2_inverse/(fold_inverse_sum)
fold3_weight = fold3_inverse/(fold_inverse_sum)
fold4_weight = fold4_inverse/(fold_inverse_sum)
fold5_weight = fold5_inverse/(fold_inverse_sum)

In [81]:
submit = pd.read_csv('../data/input/csvs/sample_submission.csv')
columns = submit.columns.values
labels = submit.columns[1:].values

weighted_ensemble = 0
for i in range(1,6):
    path = "../data/output/Train001_fold" + str(i) +"_sub.csv"
    weight = 1
    if i == 1: weight = fold1_weight
    elif i == 2: weight = fold2_weight
    elif i == 3: weight = fold3_weight
    elif i == 4: weight = fold4_weight
    elif i == 5: weight = fold5_weight
        
    weighted_ensemble += pd.read_csv(path).values[:,1:] * weight # foldの重みを掛ける

ensemble_df = pd.DataFrame(columns=columns)
ensemble_df['img'] = submit['img']
ensemble_df[labels] = weighted_ensemble

ensemble_df.to_csv("../data/output/Train001_weighted_ensemble_sub.csv", index=False)

#### 推論時のデータ拡張: TTA

In [125]:
tta_num = 3

tta_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20, # rotation range
    width_shift_range=30/224, # 30 pixel
    height_shift_range=30/224, # 30 pixel
    zoom_range=0.2
)

tta_generator = tta_datagen.flow_from_dataframe(
    submit,
    directory='../data/input/imgs/test/',
    x_col='img',
    y_col='c0', # ダミー変数
    target_size=(224, 224),
    class_mode=None,
    batch_size=1,
    verbose=1,
    shuffle=False
)

Found 79726 validated image filenames.


In [127]:
import tqdm 
submit = pd.read_csv('../data/input/csvs/sample_submission.csv')
columns = submit.columns.values
labels = submit.columns[1:].values

# len(test_generator)

def inference(fold_num):
    print(f"Fold {fold_num} inference")
    model = create_model()
    weight_path = '../model/Train001_fold' + str(fold_num) + '_best_model.h5'
    output_path = '../data/output/Train001_TTA_fold' + str(fold_num) + '_sub.csv'
    model.load_weights(weight_path)
    
    # Test time augumentation: 3 times
    preds = 0
    for i in range(tta_num):
        print(f"TTA {i+1}")
        preds += model.predict(tta_generator, verbose=1) / tta_num 

#     pred = model.predict(test_generator, verbose=1)
    pred_df = pd.DataFrame(columns=columns)
    pred_df['img'] = submit['img']
    pred_df[labels] = preds
    pred_df.to_csv(output_path, index=False)

In [128]:
inference(1)
inference(2)
inference(3)
inference(4)
inference(5)

Fold 1 inference
TTA 1
79726/79726 [==============================] - 2565s 32ms/step
TTA 2
79726/79726 [==============================] - 2072s 26ms/step
TTA 3
79726/79726 [==============================] - 2182s 27ms/step
Fold 2 inference
TTA 1
79726/79726 [==============================] - 2118s 27ms/step
TTA 2
79726/79726 [==============================] - 2177s 27ms/step
TTA 3
79726/79726 [==============================] - 2289s 29ms/step
Fold 3 inference
TTA 1
79726/79726 [==============================] - 2287s 29ms/step
TTA 2
79726/79726 [==============================] - 2255s 28ms/step
TTA 3
79726/79726 [==============================] - 2289s 29ms/step
Fold 4 inference
TTA 1
79726/79726 [==============================] - 2244s 28ms/step
TTA 2
79726/79726 [==============================] - 1913s 24ms/step
TTA 3
79726/79726 [==============================] - 1939s 24ms/step
Fold 5 inference
TTA 1
79726/79726 [==============================] - 2211s 28ms/step
TTA 2
79726/79726 

#### TTA アンサンブル：単純平均

In [129]:
submit = pd.read_csv('../data/input/csvs/sample_submission.csv')
columns = submit.columns.values
labels = submit.columns[1:].values

ensemble = 0
for i in range(1,6):
    path = "../data/output/Train001_TTA_fold" + str(i) +"_sub.csv"
    ensemble += pd.read_csv(path).values[:,1:] / 5 # fold数で割る

ensemble_df = pd.DataFrame(columns=columns)
ensemble_df['img'] = submit['img']
ensemble_df[labels] = ensemble

ensemble_df.to_csv("../data/output/Train001_TTA_ensemble_sub.csv", index=False)

#### TTA アンサンブル：加重平均

fold数ごとのPublic scoreを元に加重平均  
  
 fold: Public score  
 fold1: 0.56271  
 fold2: 0.51018  
 fold3: 0.58452  
 fold4: 0.53186  
 fold5: 0.62222  

In [138]:
fold1_weight = 0.2
fold2_weight = 0.25
fold3_weight = 0.2
fold4_weight = 0.25
fold5_weight = 0.1

In [136]:
submit = pd.read_csv('../data/input/csvs/sample_submission.csv')
columns = submit.columns.values
labels = submit.columns[1:].values

weighted_ensemble = 0
for i in range(1,6):
    path = "../data/output/Train001_TTA_fold" + str(i) +"_sub.csv"
    weight = 1
    if i == 1: weight = fold1_weight
    elif i == 2: weight = fold2_weight
    elif i == 3: weight = fold3_weight
    elif i == 4: weight = fold4_weight
    elif i == 5: weight = fold5_weight
        
    weighted_ensemble += pd.read_csv(path).values[:,1:] * weight # foldの重みを掛ける

ensemble_df = pd.DataFrame(columns=columns)
ensemble_df['img'] = submit['img']
ensemble_df[labels] = weighted_ensemble

ensemble_df.to_csv("../data/output/Train001_TTA_weighted_ensemble_sub.csv", index=False)